[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sensioai/blog/blob/master/028_pytorch_nn/pytorch_nn.ipynb)

# Pytorch - Redes Neuronales

En el post [anterior](https://sensioai.com/blog/027_pytorch_intro) hicimos una introducción al framework de `redes neuronales` `Pytorch`. Hablamos de sus tres elementos fundamentales: el objeto `tensor` (similar al `array` de `NumPy`) `autograd` (que nos permite calcular derivadas de manera automáticas) y el soporte GPU. En este post vamos a entrar en detalle en la  funcionalidad que nos ofrece la librería para diseñar redes neuronales de manera flexible.

In [1]:
import torch

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder


## Modelos secuenciales

La forma más sencilla de definir una `red neuronal` en `Pytorch` es utilizando la clase `Sequentail`. Esta clase nos permite definir una secuencia de capas, que se aplicarán de manera secuencial (las salidas de una capa serán la entrada de la siguiente). Ésto ya lo conocemos de posts anteriores, ya que es la forma ideal de definir un `Perceptrón Multicapa`.

In [2]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)

# D_in, H1, H2, D_out = 784, 100, 50, 10
# model = torch.nn.Sequential(
#     torch.nn.Linear(D_in, H1),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H1, H2),
#     torch.nn.ReLU(),
#     torch.nn.Linear(H2, D_out),
# )


El modelo anterior es un `MLP` con 784 entradas, 100 neuronas en la capa oculta y 10 salidas. Podemos usar este modelo para hacer un clasificador de imágenes con el dataset MNIST. Pero primero, vamos a ver como podemos calcular las salidas del modelo a partir de unas entradas de ejemplo.

In [3]:
outputs = model(torch.randn(64, 784))
outputs.shape

torch.Size([64, 10])

In [4]:
print(outputs[0][:])

tensor([-0.0320, -0.0143,  0.1334, -0.3416, -0.4030,  0.2088,  0.3316,  0.1208,
        -0.1692, -0.4687], grad_fn=<SliceBackward0>)


Como puedes ver, simplemente le pasamos los inputs al modelo (llamándolo como una función). En este caso, usamos un tensor con 64 vectores de 784 valores. Es importante remarcar que los modelos de `Pytorch` (por lo general) siempre esperan que la primera dimensión sea la dimensión *batch*. Si queremos entrenar esta red en una GPU, es tan sencillo como

In [5]:
model.to("cuda")

Sequential(
  (0): Linear(in_features=784, out_features=100, bias=True)
  (1): ReLU()
  (2): Linear(in_features=100, out_features=10, bias=True)
)

Vamos a ver ahora como entrenar este modelo con el dataset MNIST.

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#from sklearn.datasets import fetch_openml

# descarga datos

mnist = pd.read_csv("/content/drive/MyDrive/Inteligencia Artificial 2/Datasets/Dig-MNIST.csv")

#X, Y = mnist["data"], mnist["target"]
#X.shape, Y.shape

# Obtener características (píxeles) y etiquetas (números)
X = mnist.iloc[:, 1:].values  # Seleccionar todas las columnas excepto la primera como características
Y = mnist.iloc[:, 0].values   # Seleccionar la primera columna como etiquetas

# Verificar las formas de X e Y
print("Forma de X:", X.shape)
print("Forma de Y:", Y.shape)

Forma de X: (10240, 784)
Forma de Y: (10240,)


In [8]:
import numpy as np

# normalización y split
import numpy as np
x_2=np.array(X)
y_2=np.array(Y)

# normalización y split

X_train =x_2[:8000] / 255.
X_test =x_2[8000:] / 255.
y_train = y_2[:8000].astype(np.int32)
y_test = y_2[8000:].astype(np.int32)



# X_train, X_test, y_train, y_test = X[:60000] / 255., X[60000:] / 255., Y[:60000].astype(np.float32), Y[60000:].astype(np.float32)

In [9]:
# función de pérdida y derivada

def softmax(x):
    return torch.exp(x) / torch.exp(x).sum(axis=-1,keepdims=True)

def cross_entropy(output, target):
    logits = output[torch.arange(len(output)), target]
    loss = - logits + torch.log(torch.sum(torch.exp(output), axis=-1))
    loss = loss.mean()
    return loss

In [10]:
# X_train

In [11]:
torch.cuda.is_available()

True

In [12]:
print(X)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
# convertimos datos a tensores y copiamos en gpu

X_t = torch.from_numpy(X_train).float().cuda()
Y_t = torch.from_numpy(y_train).long().cuda()

# bucle entrenamiento
epochs = 1000
lr = 0.8
log_each = 100
l = []
for e in range(1, epochs + 1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = cross_entropy(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    model.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    with torch.no_grad():
        for param in model.parameters():
            param -= lr * param.grad

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

Epoch 100/1000 Loss 0.74265
Epoch 200/1000 Loss 0.53530
Epoch 300/1000 Loss 0.42603
Epoch 400/1000 Loss 0.35249
Epoch 500/1000 Loss 0.29899
Epoch 600/1000 Loss 0.25860
Epoch 700/1000 Loss 0.22730
Epoch 800/1000 Loss 0.20249
Epoch 900/1000 Loss 0.18245
Epoch 1000/1000 Loss 0.16596


Como puedes observar en el ejemplo, podemos calcular la salida del modelo con una simple línea. Luego calculamos la función de pérdida, y llamando a la función `backward` `Pytorch` se encarga de calcular las derivadas de la misma con respecto a todos los parámetros del modelo automáticamente (si no queremos acumular estos gradientes, nos aseguramos de llamar a la función `zero_grad` para ponerlos a cero antes de calcularlos). Por útlimo, podemos iterar por los parámetros del modelo aplicando la regla de actualización deseada (en este caso usamos `descenso por gradiente`).

In [14]:
from sklearn.metrics import accuracy_score

def evaluate(x):
    model.eval()
    y_pred = model(x)
    y_probas = softmax(y_pred)
    return torch.argmax(y_probas, axis=1)

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

0.8816964285714286

Existen algunos tipos de capas que se comportan diferente en función de si estamos entrenando la red o usándola para generar predicciones. Podemos controlar el modo en el que queremos que esté nuestra red con las funciones `train` y `eval`.

## Optimizadores y Funciones de pérdida

En el ejemplo anterior hemos calculado la función de pérdida y aplicado la regla de optimización de forma manual. Sin embargo, `Pytorch` nos ofrece funcionalidad que nos abstrae estos cálculos ofreciendo además flexibilidad para aplicar diferentes funciones de pérdida o algoritmos de optimización de manera sencilla. Podemos encontrar diferentes funciones de pérdida ya implementadas en el paquete `torch.nn`.

In [15]:
criterion = torch.nn.CrossEntropyLoss()

Mientras que los optimizadores se encuentran en el paquete `torch.optim`

In [16]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

Puedes ver la lista completa de funciones de pérdida y optimizadores disponibles en la [documentación](https://pytorch.org/docs/stable/index.html), aunque como ya has visto siempre puedes definir los tuyos propios fácilmente.

Una vez definidos estos dos objetos, nuestro bucle de entrenamiento se simplifica considerablemente.

In [17]:
D_in, H, D_out = 784, 100, 10

model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
).to("cuda")

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.75849
Epoch 20/100 Loss 1.51942
Epoch 30/100 Loss 1.27767
Epoch 40/100 Loss 1.11866
Epoch 50/100 Loss 1.01043
Epoch 60/100 Loss 0.92837
Epoch 70/100 Loss 0.86763
Epoch 80/100 Loss 0.81892
Epoch 90/100 Loss 0.77679
Epoch 100/100 Loss 0.74104


0.8357142857142857

## Modelos custom

Si bien en muchos casos definir una `red neuronal` como una secuencia de capas es suficiente, en otros casos será un factor limitante. Un ejemplo son las redes residuales, en las que no sólo utilizamos la salida de una capa para alimentar la siguiente si no que, además, le sumamos su propia entrada. Este tipo de arquitectura no puede ser definida con la clase `Sequential`, y para ello necesitamos hacer un modelo *customizado*. Para ello, `Pytroch` nos ofrece la siguiente sintaxis.

In [18]:
# creamos una clase que hereda de `torch.nn.Module`

class ModeloPersonalizado(torch.nn.Module):

    # constructor
    def __init__(self, D_in, H, D_out):

        # llamamos al constructor de la clase madre
        super(ModeloPersonalizado, self).__init__()

        # definimos nuestras capas
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    # lógica para calcular las salidas de la red
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

En primer lugar, necesitamos definir una nueva clase que herede de la clase `torch.nn.Module`. Esta clase madre aportará toda la funcionalidad esencial que necesita una `red neuronal` (soporte GPU, iterar por sus parámeteros, etc). Luego, en esta clase necesitamos definir mínimos dos funciones:

- `init`: en el constructor llamaremos al constructor de la clase madre y después definiremos todas las capas que querramos usar en la red.
- `forward`: en esta función definimos toda la lógica que aplicaremos desde que recibimos los inputs hasta que devolvemos los outputs.

En el ejemplo anterior simplemente hemos replicado la misma red (puedes conseguir el mismo efecto usando la clase `Sequential`).

In [19]:
model = ModeloPersonalizado(784, 100, 10)
# Codigo para saber si el modelo esta votando los datos en las cantidades correctas
x_prueba = torch.randn(64, 784)
print(x_prueba)
outputs = model(x_prueba)
outputs.shape

tensor([[-0.7975,  0.4020, -0.0529,  ...,  0.2093,  0.0839, -0.5453],
        [ 0.3251, -1.3477,  0.1486,  ...,  0.2561,  0.5434, -0.0507],
        [ 0.2910,  0.4904, -1.4660,  ..., -1.5387, -0.0908, -1.9974],
        ...,
        [ 1.2251,  1.4736,  0.4680,  ..., -2.1641, -0.1409, -1.3830],
        [-0.3671, -1.0128,  1.1067,  ..., -1.3502,  0.7997,  1.3077],
        [-1.6358, -0.3203,  0.9018,  ..., -1.3204, -0.0964,  0.9090]])


torch.Size([64, 10])

Ahora, podemos entrenar nuestra red de la misma forma que lo hemos hecho anteriormente.

In [20]:
model.to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.8)

epochs = 100
log_each = 10
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 10/100 Loss 1.72759
Epoch 20/100 Loss 1.46506
Epoch 30/100 Loss 1.25185
Epoch 40/100 Loss 1.10039
Epoch 50/100 Loss 0.99366
Epoch 60/100 Loss 0.91643
Epoch 70/100 Loss 0.85574
Epoch 80/100 Loss 0.80673
Epoch 90/100 Loss 0.76594
Epoch 100/100 Loss 0.73114


0.8308035714285714

Aquí puedes ver otro ejemplo de como definir un `MLP` con conexiones residuales, algo que no podemos hacer simplemente usando un modelo secuencial.

In [21]:
class ModelCustom2(torch.nn.Module):

    def __init__(self, D_in, H, D_out):
        super(ModelCustom2, self).__init__()
        self.fc1 = torch.nn.Linear(D_in, H)
        self.relu = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(H, D_out)

    def forward(self, x):
        x1 = self.fc1(x)
        x = self.relu(x1)
        x = self.fc2(x + x1)
        return x

In [33]:
model = ModelCustom2(784, 1000, 100).to("cuda")
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.2)

epochs = 1000
log_each = 100
l = []
model.train()
for e in range(1, epochs+1):

    # forward
    y_pred = model(X_t)

    # loss
    loss = criterion(y_pred, Y_t)
    l.append(loss.item())

    # ponemos a cero los gradientes
    optimizer.zero_grad()

    # Backprop (calculamos todos los gradientes automáticamente)
    loss.backward()

    # update de los pesos
    optimizer.step()

    if not e % log_each:
        print(f"Epoch {e}/{epochs} Loss {np.mean(l):.5f}")

y_pred = evaluate(torch.from_numpy(X_test).float().cuda())
accuracy_score(y_test, y_pred.cpu().numpy())

Epoch 100/1000 Loss 0.83160
Epoch 200/1000 Loss 0.65934
Epoch 300/1000 Loss 0.57929
Epoch 400/1000 Loss 0.52728
Epoch 500/1000 Loss 0.48784
Epoch 600/1000 Loss 0.45597
Epoch 700/1000 Loss 0.42853
Epoch 800/1000 Loss 0.40555
Epoch 900/1000 Loss 0.38300
Epoch 1000/1000 Loss 0.36265


0.8455357142857143

De esta manera, tenemos mucha flexibilidad para definir nuestras redes.

## Accediendo a las capas de una red

En ocasiones queremos acceder a una capa en particular de nuestra red. Para ello, podemos acceder utilizando su nombre.

In [34]:
model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=1000, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=1000, out_features=100, bias=True)
)

In [35]:
model.fc1

Linear(in_features=784, out_features=1000, bias=True)

También podemos acceder directamente a los tensores que contienen los parámetros con las propiedades adecuadas

In [36]:
model.fc1.weight

Parameter containing:
tensor([[-0.0240, -0.0170,  0.0186,  ..., -0.0323, -0.0330,  0.0239],
        [-0.0343,  0.0234, -0.0025,  ...,  0.0221,  0.0239,  0.0194],
        [-0.0074, -0.0105,  0.0272,  ...,  0.0262,  0.0007, -0.0081],
        ...,
        [-0.0058,  0.0116,  0.0242,  ...,  0.0322,  0.0009,  0.0165],
        [ 0.0114, -0.0101,  0.0169,  ..., -0.0071,  0.0196, -0.0298],
        [-0.0342,  0.0331, -0.0042,  ...,  0.0179, -0.0179,  0.0059]],
       device='cuda:0', requires_grad=True)

In [37]:
model.fc1.bias

Parameter containing:
tensor([-5.5942e-03,  4.1684e-02,  2.0842e-02, -2.3471e-02,  2.7334e-02,
         4.1995e-02,  1.6146e-02,  6.7494e-04, -2.2498e-02,  6.7495e-02,
         3.5134e-02, -5.1514e-02, -2.2364e-02,  6.2335e-02, -1.8160e-03,
        -1.0916e-01,  5.8423e-02,  2.0487e-02,  1.0419e-01, -1.4122e-02,
        -1.9191e-02, -1.5970e-01, -2.9123e-02, -2.6600e-02,  1.0052e-01,
        -7.9297e-02, -5.5097e-02,  5.6630e-03,  7.1414e-03, -7.0787e-03,
        -6.7167e-02, -9.2235e-02,  6.3453e-02,  3.7388e-02, -8.6513e-02,
        -9.7600e-02,  3.0909e-02, -1.5245e-02,  3.8811e-02, -1.4016e-02,
         3.1801e-03, -3.9992e-02,  3.6655e-02, -4.4679e-03, -2.6743e-02,
         8.1266e-03,  8.5863e-02,  9.1428e-03,  2.2186e-02,  5.3463e-02,
         2.1230e-02,  3.2582e-02, -5.6709e-03,  2.6060e-02, -2.5412e-03,
        -4.1327e-02, -1.1163e-03, -1.9972e-02,  1.0996e-01, -1.9625e-02,
         9.2916e-04, -1.1607e-01,  5.0887e-02, -2.1630e-02, -5.2308e-02,
         1.4197e-02,  1.2032e

Es posible sobreescribir una capa de la siguiente manera

In [38]:
model.fc2 = torch.nn.Linear(100, 1)

model

ModelCustom2(
  (fc1): Linear(in_features=784, out_features=1000, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=100, out_features=1, bias=True)
)

Ahora, la capa final de nuestra red tiene solo una salida. Esta nueva capa ha sido inicializada de manera aleatoria, por lo que esta nueva red no nos va a servir de mucho. Sin embargo, podríamos volver a entrenar esta red en otro problema en el que solo necesitemos una salida aprovechando los pesos que ya hemos entrenado anteriormente con el dataset MNIST. Esto es la base del *transfer learning*, una técnica que utilizaremos muchísimo más adelante y la cual explicaremos en detalle.

A continuación encontrarás varios trucos a la hora de crear redes neuronales a partir de otras que te pueden resultar útiles.

In [39]:
# obtener una lista con las capas de una red

list(model.children())

[Linear(in_features=784, out_features=1000, bias=True),
 ReLU(),
 Linear(in_features=100, out_features=1, bias=True)]

In [40]:
# crear nueva red a partir de la lista (excluyendo las útlimas dos capa)

new_model = torch.nn.Sequential(*list(model.children())[:-2])
new_model

Sequential(
  (0): Linear(in_features=784, out_features=1000, bias=True)
)

In [41]:
# crear nueva red a partir de la lista (excluyendo las útlima capa)

new_model = torch.nn.ModuleList(list(model.children())[:-1])
new_model

ModuleList(
  (0): Linear(in_features=784, out_features=1000, bias=True)
  (1): ReLU()
)

## Resumen

En este post hemos visto la funcionalidad que `Pytorch` nos ofrece a la hora de definir y entrenar nuestras `redes neuronales`. El paquete `torch.nn` contiene todo lo necesario para diseñar nuestros modelos, ya sea de manera secuencial o con una clase *custom* para arquitecturas más complicadas. También nos da muchas funciones de pérdida que podemos usar directamente para entrenar las redes. Te recomiendo encarecidamente que le eches un vistazo a la [documentación](https://pytorch.org/docs/stable/nn.html) par hacerte una idea de todo lo que puedes hacer. También hemos visto como el paquete `torch.optim` nos oferece algoritmos de optimización que también nos hacen la vida más fácil a la hora de entrenar nuestras redes.